<a href="https://colab.research.google.com/github/kazaros91/etherscan_verified_contracts/blob/master/BART_cross_domain_doctor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependendcies

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 1.8MB 16.8MB/s 
     |████████████████████████████████| 2.9MB 50.6MB/s 
     |████████████████████████████████| 890kB 60.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a8a0920e7e2bfe507da54820a9824a1cf5dc15e067d3b5c04d358ee3fa6faae1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!nvidia-smi

Mon Jan 25 21:10:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'2012-2013秋季学期考试安排 (1).gsheet'
 2012-2013秋季学期考试安排.gsheet
 2_5434100138815523315.pdf
 3_4.jpg
 403319IELUVN0988-010219_10276750.xls
 536234TBSLLU3777-011018_9063096.xls
 best.pt
 canc.doc
'CapsNet0 (1).gdoc'
'CapsNet0 (2).gdoc'
'CapsNet0 (3).gdoc'
'CapsNet0 (4).gdoc'
 CapsNet0.gdoc
 Capsule-untrustworthy_review_detection.zip
'cen2019 (1).pdf'
 cen2019.pdf
'Colab Notebooks'
'Computer Model Database (Marvel).gsheet'
 CVArshaluysArzumanyan.pdf
 dataset_op_spam_v1.4.zip
 datasets
 deception_dataset.zip
'[Electronics] Manuscript ID_ electronics-913533 - APC Invoice.eml'
'Employees-list_new (1).gsheet'
 Employees-list_new.gsheet
 Employees-list_new.xlsx
 exp2
'Figures 7-9.gsheet'
 IMG_20201225_021058.jpg
'IMG_20201225_232437 (1).jpg'
 IMG_20201225_232437.jpg
 IMG_20201225_232452.jpg
'Improve Reputation Evaluation of Crowdsourcing Participants Using Multidimensional Index and Machine Learning Techniques (1).pdf'
'Improve Reputation Evaluation of Crowdsourcing Partici

In [ ]:
!cp /mydrive/deception_dataset.zip /content/
!unzip deception_dataset.zip
!rm deception_dataset.zip
!rm -r __MACOSX

Streaming output truncated to the last 5000 lines.
  inflating: deception_dataset/restaurant/deceptive_MTurk/85.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._85.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/153.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._153.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/147.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._147.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/146.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._146.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/152.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._152.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/84.txt  
  inflating: __MACOSX/deception_dataset/restaurant/deceptive_MTurk/._84.txt  
  inflating: deception_dataset/restaurant/deceptive_MTurk/90.txt  
  in

In [ ]:
import numpy as np
import string
import os
import io
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


SEQUENCE_LEN = 200


def read_from_dir(in_fold_dir):
    reviews = []
    for filename in os.listdir(in_fold_dir):
        if filename.endswith(".txt"):
            # read from file containing the text review
            in_filepath = in_fold_dir + filename
            with io.open(in_filepath, 'r', encoding='windows-1252') as f:
                review = f.read()
                # if len(review) > SEQUENCE_LEN:
                #     review = review[:200]
                reviews.append(review)    

    # print("number of reviews: {}, name: {}".format(len(reviews), in_fold_dir)) 
    print(reviews[:2])         
    return reviews


def load_data():
    trustworthy_hotel_positive = read_from_dir('deception_dataset/hotel/positive/truthful/') 
    trustworthy_hotel_negative = read_from_dir('deception_dataset/hotel/negative/truthful/')
    trustworthy_restaurant = read_from_dir('deception_dataset/restaurant/truthful/')
    trustworthy_doctor = read_from_dir('deception_dataset/doctor/truthful/')[:200]

    untrustworthy_hotel_positive = read_from_dir('deception_dataset/hotel/positive/deceptive_turker/')
    untrustworthy_hotel_negative = read_from_dir('deception_dataset/hotel/negative/deceptive_turker/') 
    untrustworthy_hotel_expert_positive =  read_from_dir('deception_dataset/hotel/positive/deceptive_expert/')
    untrustworthy_hotel_expert_negative =  read_from_dir('deception_dataset/hotel/negative/deceptive_expert/')
    
    untrustworthy_restaurant = read_from_dir('deception_dataset/restaurant/deceptive_MTurk/')[:200]
    untrustworthy_doctor = read_from_dir('deception_dataset/doctor/deceptive_MTurk/')

    TRUSTWORTHY_REVIEWS_DICT = {}
    UNTRUSTWORTHY_REVIEWS_DICT = {}

    TRUSTWORTHY_REVIEWS_DICT["cross_domain"] = {
        "hotel":trustworthy_hotel_positive + trustworthy_hotel_negative,
        "restaurant":trustworthy_restaurant,
        "doctor":trustworthy_doctor
    }

    UNTRUSTWORTHY_REVIEWS_DICT["cross_domain"] = { 
        "hotel":untrustworthy_hotel_positive + untrustworthy_hotel_negative,
        "restaurant":untrustworthy_restaurant,
        "doctor":untrustworthy_doctor[:200]
    }

    TRUSTWORTHY_REVIEWS_DICT["mixed_domain"] = {
        "hotel":trustworthy_hotel_positive + trustworthy_hotel_negative,
        "restaurant":trustworthy_restaurant,
        "doctor":trustworthy_doctor
    }

    UNTRUSTWORTHY_REVIEWS_DICT["mixed_domain"] = { 
        "hotel":untrustworthy_hotel_positive + untrustworthy_hotel_expert_positive + untrustworthy_hotel_negative + untrustworthy_hotel_expert_negative,
        "restaurant":untrustworthy_restaurant,
        "doctor":untrustworthy_doctor
    }

    return TRUSTWORTHY_REVIEWS_DICT, UNTRUSTWORTHY_REVIEWS_DICT


TRUSTWORTHY_REVIEWS_DICT, UNTRUSTWORTHY_REVIEWS_DICT = load_data()

["We have been for the first time in Chicago and stayed in the Swissotel for five nights,a wonderful place.Room was on the 29th floor,absolutely clean and nice and a breathtaking lakeview.Breakfast perfect and service really good.People were always helpful.It's a little bit far from Magnificent mile and the loop,but in front of the hotel there are buses (number6)waiting there next turn,not a real bus stop,but drivers are frindly and you can get on the bus.Next time going to Chicago we will again stay in the Swissotel. \n", "Stayed at the InterContinental for an entire week on business and for the most part a very elegant, convenient visit. Stayed in the historic tower. Very nice rooms, great view, attentive staff, convenient location just north of the Tribune Tower & the river, and nice fitness club at $16 for the entire stay. The only downside is I couldn't get over the feeling that I was being nickle & dimed: $10 a day for Wi-Fi, $6 bottles of water in the room, $26 mini-bottle of wi

In [ ]:
def get_reviews_from_dict(reviews_dict, keys = []):
    if not keys:
        raise Exception("Invalid key: key must be one of the following: \'hotel\', \'restaurant\', \'doctor\'")

    reviews = []
    for key in keys:
        if key in reviews_dict:
            reviews += reviews_dict[key]
        else:
            raise Exception("Invalid key: key must be one of the following: \'hotel\', \'restaurant\', \'doctor\'")
    return reviews

def get_reviews(review_type="", keys = [], data_name="cross_domain"):
    if not review_type:
        raise Exception("Invalid review_type: review_type must be either \'trustworthy\' or \'untrustworthy\'.")
    if not keys:
        raise Exception("Invalid key: key must be one of the follwowing: \'hotel\', \'restaurant\', \'doctor\'")

    if review_type=="trustworthy":
        return get_reviews_from_dict(reviews_dict=TRUSTWORTHY_REVIEWS_DICT[data_name], keys=keys)
    elif review_type=="untrustworthy":
        return get_reviews_from_dict(reviews_dict=UNTRUSTWORTHY_REVIEWS_DICT[data_name], keys=keys)
    else:
        raise Exception("Invalid review_type: review_type must be either \'trustworthy\' or \'untrustworthy\'.")


def get_data(keys=[], data_name="cross_domain"):
    if data_name != "cross_domain" and data_name != "mixed_domain":
        raise Exception("Invalid dataset name")

    trustworthy_reviews   = get_reviews(review_type="trustworthy", keys=keys, data_name=data_name)
    untrustworthy_reviews = get_reviews(review_type="untrustworthy", keys=keys, data_name=data_name)
    
    reviews = trustworthy_reviews + untrustworthy_reviews

    # generate labels
    labels_trustworthy   = [[1,0] for _ in range(len(trustworthy_reviews   ))]
    labels_untrustworthy = [[0,1] for _ in range(len(untrustworthy_reviews ))]
    labels = labels_trustworthy + labels_untrustworthy

    print("dataset_name: {}".format(data_name+"_"+keys[0]))
    print("trustworthy_reviews: {}".format(len(trustworthy_reviews)))
    print("untrustworthy_reviews: {}".format(len(untrustworthy_reviews)))
    print("reviews len: ", len(reviews))
    print("labels len: ", len(labels))
    print()
        
    return reviews, labels

# Data Helpers

In [ ]:
import numpy as np
import os
import re
import itertools
import scipy.sparse as sp
from collections import Counter
import torch


SEQUENCE_LEN = 200


def build_vocab(sentences, vocab_size=50000):
    word_counts = Counter(itertools.chain(*sentences))
    vocabulary_inv = [x[0] for x in word_counts.most_common(vocab_size)]
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    # append <UNK/> symbol to the vocabulary
    vocabulary['<UNK/>'] = len(vocabulary)
    vocabulary_inv.append('<UNK/>')
    return [vocabulary, vocabulary_inv]


def build_input_data(sentences, vocabulary):
    x = np.array([[vocabulary[word] if word in vocabulary else vocabulary['<UNK/>'] for word in sentence] for sentence in sentences])
    return x


def load_data(fold = 1, vocab_size=30001):
    # Load data
    trustworthy_reviews_for_training, trustworthy_reviews_for_testing, untrustworthy_reviews_for_training, untrustworthy_reviews_for_testing = data.load_data(fold=fold)
    train = trustworthy_reviews_for_training + untrustworthy_reviews_for_training
    test  = trustworthy_reviews_for_testing + untrustworthy_reviews_for_testing

    # generate labels
    train_labels1 = [[1,0] for _ in range(len(trustworthy_reviews_for_training   ))]
    train_labels0 = [[0,1] for _ in range(len(untrustworthy_reviews_for_training ))]
    test_labels1  = [[1,0] for _ in range(len(trustworthy_reviews_for_testing    ))]
    test_labels0  = [[0,1] for _ in range(len(untrustworthy_reviews_for_testing  ))]

    train_labels = np.concatenate((np.array(train_labels1), np.array(train_labels0)), axis=0)
    test_labels  = np.concatenate((np.array(test_labels1),  np.array(test_labels0)),  axis=0)

    # convert word2idx
    vocabulary, vocabulary_inv = build_vocab(train + test, vocab_size=vocab_size)
    train = build_input_data(train, vocabulary)
    test = build_input_data(test,   vocabulary)
    
    return train, train_labels, train_labels, train, test_labels, test_labels, vocabulary, vocabulary_inv


def batch_iter(data, batch_size, num_epochs):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        shuffle_indices = np.random.permutation(np.arange(data_size))
        shuffled_data = data[shuffle_indices]
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [ ]:
def get_cross_domain_dataset(test_keys=[]):
    train_keys = ["hotel"]

    train_reviews, train_labels = get_data(keys=train_keys, data_name="cross_domain")
    test_reviews,  test_labels  = get_data(keys=test_keys, data_name="cross_domain")
    all_reviews = train_reviews + test_reviews
    
    return train_reviews, train_labels, test_reviews, test_labels

def get_cross_domain_dataset_restaurant():
    return get_cross_domain_dataset(test_keys=["restaurant"])

def get_cross_domain_doctor_doctor():
    return get_cross_domain_dataset(test_keys=["doctor"])

In [ ]:
train_data, train_labels, test_data, test_labels = get_cross_domain_dataset(test_keys=["restaurant"])
print("cross_domain_dataset_restaurant_train len: ", len(train_data))
print("cross_domain_dataset_restaurant_test len: ", len(test_data))

dataset_name: cross_domain_hotel
trustworthy_reviews: 800
untrustworthy_reviews: 800
reviews len:  1600
labels len:  1600

dataset_name: cross_domain_restaurant
trustworthy_reviews: 200
untrustworthy_reviews: 200
reviews len:  400
labels len:  400

cross_domain_dataset_restaurant_train len:  1600
cross_domain_dataset_restaurant_test len:  400


In [ ]:
train_data, train_labels, test_data, test_labels = get_cross_domain_dataset(test_keys=["doctor"])
print("cross_domain_dataset_doctor_train len: ", len(train_data))
print("cross_domain_dataset_doctor_test len: ", len(test_data))
train_data[:10]

dataset_name: cross_domain_hotel
trustworthy_reviews: 800
untrustworthy_reviews: 800
reviews len:  1600
labels len:  1600

dataset_name: cross_domain_doctor
trustworthy_reviews: 200
untrustworthy_reviews: 200
reviews len:  400
labels len:  400

cross_domain_dataset_doctor_train len:  1600
cross_domain_dataset_doctor_test len:  400


["We have been for the first time in Chicago and stayed in the Swissotel for five nights,a wonderful place.Room was on the 29th floor,absolutely clean and nice and a breathtaking lakeview.Breakfast perfect and service really good.People were always helpful.It's a little bit far from Magnificent mile and the loop,but in front of the hotel there are buses (number6)waiting there next turn,not a real bus stop,but drivers are frindly and you can get on the bus.Next time going to Chicago we will again stay in the Swissotel. \n",
 "Stayed at the InterContinental for an entire week on business and for the most part a very elegant, convenient visit. Stayed in the historic tower. Very nice rooms, great view, attentive staff, convenient location just north of the Tribune Tower & the river, and nice fitness club at $16 for the entire stay. The only downside is I couldn't get over the feeling that I was being nickle & dimed: $10 a day for Wi-Fi, $6 bottles of water in the room, $26 mini-bottle of w

In [ ]:
def get_mix_domain_dataset(fold=1):
    keys = ["hotel", "restaurant", "doctor"]
    data, labels = get_data(keys=keys, data_name="mixed_domain")

    MAX_FOLD = 5
    LEN_DATA = len(data)
    DELTA = int(LEN_DATA / MAX_FOLD)
    start_idx = (fold-1)*DELTA
    end_idx   = start_idx + DELTA

    # np.concatenate
    data[0:start_idx, :]
    train_reviews   = np.concatenate((data[0:start_idx, :],  data[end_idx:,:]))
    train_labels = np.concatenate((labels[0:start_idx:end_idx,:], labels[end_idx:,:]))

    test_reviews   = data[start_idx:end_idx]
    test_labels = labels[start_idx:end_idx]

    return train_reviews, train_labels, test_reviews, test_labels

In [ ]:
train_data, train_labels, test_data, test_labels = get_mix_domain_dataset()

print("mixed_domain_dataset train: ", len(train_data))
print("mixed_domain_dataset test: ", len(test_data))

dataset_name: mixed_domain_hotel
trustworthy_reviews: 1200
untrustworthy_reviews: 1637
reviews len:  2837
labels len:  2837



TypeError: ignored

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings  = tokenizer(train_data, truncation=True, padding=True)
test_encodings   = tokenizer(test_data, truncation=True, padding=True)
# print(train_encodings)
# print(test_encodings['ids'][:10])

In [ ]:
import torch

class UntrustworthyReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels    = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset  = UntrustworthyReviewDataset(train_encodings, train_labels)
valid_dataset   = UntrustworthyReviewDataset(test_encodings, test_labels)

train_dataset[:11]['input_ids'].shape[1]

In [ ]:
from transformers import AutoModel
BERT = AutoModel.from_pretrained('bert-base-uncased')

# Freeze BERT Parameters

In [ ]:
# freeze all the parameters
for param in BERT.parameters():
    param.requires_grad = False

# Define Transformer-based Architecture for Fine-tuning

In [ ]:
import torch.nn as nn
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, input_ids, attention_mask):

      #pass the inputs to the model  
      x = self.bert(input_ids, attention_mask=attention_mask)
      # print(x[0])
      
      x = self.fc1(x[0])

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
model = BERT_Arch(BERT)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
model.to(device)
model.train()

# Define Optimizers

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

# Define training method

In [ ]:
# function to train the model
BATCH_SIZE = 21 # 48
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

def train(model=model):
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step, batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    input_ids      = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels         = batch['labels'].to(device)

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(input_ids, attention_mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # print('train_loss ', loss.item())

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds = preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of cla
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

Define evaluating method

In [ ]:
# function for evaluating the model
# function to train the model
BATCH_SIZE = 21 # 48
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

def evaluate(model=model):
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step, batch in enumerate(valid_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(valid_dataloader)))

    # push the batch to gpu
    input_ids      = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels         = batch['labels'].to(device)

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(input_ids, attention_mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds, labels)

      total_loss = total_loss + loss.item()
      # print('valid_loss ', loss.item())

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(valid_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds


# Define training and evaluation method


In [ ]:
from sklearn import metrics

# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch

LR = 5e-5
EPOCHS = 32

def train_and_validate():
  best_valid_loss = 30
  for epoch in range(EPOCHS):
      
      print('\n Epoch {:} / {:}'.format(epoch + 1, EPOCHS))
      
      #train model
      train_loss, train_preds = train(model)
      
      #evaluate model
      valid_loss, valid_preds = evaluate(model)
      
      #save the best model
      
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), 'best.pt')
      
      # append training and validation loss
      train_losses.append(train_loss)
      valid_losses.append(valid_loss)
      
      print(f'\nTraining Loss: {train_loss:.3f}')
      print(f'Validation Loss: {valid_loss:.3f}')

     
      # train_preds = train_preds.round().squeeze(2)
      # train_report = metrics.classification_report(labels, train_preds)
     
      # valid_preds = valid_preds.round().squeeze(2)
      # valid_report = metrics.classification_report(labels, valid_preds)

      # # print('train_preds ', train_preds)
      # # print(train_report)

      # # print('valid_preds ', valid_preds)
      # # print(valid_report)

# Start Model Training

In [ ]:
train_and_validate()

In [ ]:
from matplotlib import pyplot

def plot_results(reports, metrics):
  stages = ['Train', 'Valid']
  for stage in stages:
    plot = new plot()
    for metric in metrics:
      pyplot.subplot(212)
      pyplot.title(stage + ' ' + metric)
      for model in models:
        pyplot.plot(reports[model][stage][metric], label=model)
      pyplot.legend()
      pyplot.show()


reports = {
    'BERT':  
    {
      'Train': BERT_train_reports,
      'Valid': BERT_valid_reports
    }
    'BART':  
    {
      'Train': BART_train_reports,
      'Valid': BART_valid_reports
    }
}

metrics = ['loss', 'precision', 'recall', 'f1-score', 'accuracy'] 
plot_results(train_reports, test_reports, metrics)

In [ ]:
train_encodings  = tokenizer(train_data, truncation=True, padding=True)
test_encodings   = tokenizer(test_data, truncation=True, padding=True)
# print(train_encodings)
# print(test_encodings['ids'][:10])

In [ ]:
import torch

class UntrustworthyReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels    = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
from transformers import AutoModel
from transformers import AutoTokenizer

def train_and_evaluate(model_name='bert-based-uncased'):
  model = AutoModel.from_pretrained(model_name)
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  
  model_architecture = transformer_architecture(model)
                                               
  train_encodings  = tokenizer(train_data, truncation=True, padding=True)
  test_encodings   = tokenizer(test_data, truncation=True, padding=True)

  train_dataset    = UntrustworthyReviewDataset(train_encodings, train_labels)
  valid_dataset    = UntrustworthyReviewDataset(test_encodings, test_labels)

  train_and_evaluate(train_dataset, valid_dataset, model)

In [ ]:
model_names = ['bert-base-uncased', 'electra', 'bart']
for model in models:
  train_and_evaluate(model)